# Relación entre las actividades extracurriculares y el rendimiento académico
**- Hipótesis:**
*Los estudiantes que participan en actividades como leer libros o practicar deportes tienen mejores resultados académicos que aquellos que pasan más tiempo viendo televisión o jugando videojuegos.*


## Selección de datos

Para el dataset `egd10`, disponemos de las siguientes variables a estudiar, divididas en dos grupos según el tipo de actividad:

Hábitos relacionados con **actividades culturales y formativas**:

- `S20B`	Actividades fuera del colegio-Escuchar música
- `S20C`	Actividades fuera del colegio-Practicar deporte
- `S20D`	Actividades fuera del colegio-Leer libros, cuentos, cómic
- `S20E`	Actividades fuera del colegio-Leer prensa o revistas
- `S20F`	Actividades fuera del colegio-Jugar con amigos
- `S22A`	Internet para-Buscar información para tus estudios
- `S22B`	Internet para-Buscar información sobre deportes
- `S22C`	Internet para-Buscar información sobre juegos
- `S22D`	Internet para-Buscar información sobre música o cine
- `S22E`	Internet para-Buscar información sobre otros temas

Hábitos relacionados con **actividades de ocio digital**:

- `S20A`	Actividades fuera del colegio-Ver televisión, vídeos o DVDS
- `S20G`	Actividades fuera del colegio-Hablar por teléfono
- `S20H`	Actividades fuera del colegio-Jugar con videojuegos o con el ordenador
- `S20I`	Actividades fuera del colegio-Participar en chats o messenger
- `S20J`	Actividades fuera del colegio-Usar internet
- `S21A`	Frecuencia utilización ordenador-En casa
- `S21C`	Frecuencia utilización ordenador-En otros lugares
- `S22F`	Internet para-Comunicarme con otras personas

También se han seleccionado las variables relacionadas con el rendimiento académico, como se ha hecho en otras hipótesis.


In [ ]:
import pandas as pd

In [ ]:
egd10 = pd.read_excel("data/EGD10.xlsx")

In [ ]:
cultural_features = [
    "S20B",
    "S20C",
    "S20D",
    "S20E",
    "S20F",
    "S22A",
    "S22B",
    "S22C",
    "S22D",
    "S22E",
]
digital_features = [
    "S20A",
    "S20G",
    "S20H",
    "S20I",
    "S20J",
    "S21A",
    "S21C",
    "S22F",
]
targets = list(map(lambda x: x.upper(), [
    "PV1ling",
    "PV2ling",
    "PV3ling",
    "PV4ling",
    "PV5ling",
    "PV1mat",
    "PV2mat",
    "PV3mat",
    "PV4mat",
    "PV5mat",
    "PV1mfis",
    "PV2mfis",
    "PV3mfis",
    "PV4mfis",
    "PV5mfis",
    "PV1syc",
    "PV2syc",
    "PV3syc",
    "PV4syc",
    "PV5syc",
]))


In [ ]:
egd10 = egd10[targets + cultural_features + digital_features]
egd10.dropna(inplace=True)


## Tratamiento y limpieza de datos


La transformación de las variables requerirá de la agrupación de las mismas, para ello tendremos que realizar previamente una unificación de las unidades de medida. Adicionalmente, para que esto sea posible, tendremos que trater primero los valores nulos de las variables que vayamos a tener en cuenta.

### Tratamiento de valores nulos

En relación con las variables seleccionadas, será necesario realizar un tratamiento específico de los valores por defecto. En todas estas variables, el valor 9 se utiliza como marcador por defecto, mientras que el valor 1 se emplea para indicar ausencia o no uso.

Por esta razón, procederemos a descartar aquellas filas en las que el valor de la variable sea 9, ya que no aportan información relevante para el análisis.


In [ ]:
egd10 = egd10[(egd10[cultural_features + digital_features] != 9).all(axis=1)]

Para el caso de las variables de `target`, procederemos de igual manera que con las anteriores hipótesis.

### Tratamiento de las variables

En primer lugar, agrupamos las variables de `target`:

In [ ]:
egd10.loc[:, "Nota_Media"] = egd10[targets].mean(axis=1)
egd10.drop(targets, axis=1, inplace=True)
egd10.loc[:, "Nota_Media"] = egd10["Nota_Media"].apply(lambda x: x / 100)

Para el caso de las variables seleccionadas, observamos en primer lugar que todas son categóricas ordinales.

Para las variables 20[A-J], la medida es tiempo en horas, aunque no queda claro si es al día o a la semana.
El máximo es de 2h, por lo que se ha decidido considerar que es al día.

En cuanto a 21[A-C] y 22[A-F], la medida es la frecuencia de uso.


Para poder unificar las variables, haremos la siguiente conversión a "tiempo dedicado", medido en horas:

Para la variable de tiempo:

- Nada de tiempo → 0 horas
- Menos de 30 minutos → 0.25 horas
- Entre 30 y 60 minutos → 0.75 horas
- Entre 1 y 2 horas → 1.5 horas
- Más de 2 horas → 2.5 horas

Para la variable de frecuencia:

- Nunca o casi nunca → 0 horas (se puede tratar como "0 tiempo dedicado").
- Una o dos veces al mes → 0.2 horas (aproximadamente equivalente a dedicar menos de 30 minutos en promedio semanal).
- Una o dos veces a la semana → 0.65 horas (aproximadamente 30-60 minutos a la semana).
- Todos o casi todos los días → 1.25 horas (equivalente a dedicar una hora y pico diaria en promedio).

Lo que haremos es mapear la conversión a cada una de las variables y después haremos un total agrupando en "actividades de ocio digital" o "actividades culturales y formativas":

In [ ]:
# Mapeos para las variables de tiempo y frecuencia
# Los valores representan horas estimadas para cada categoría
map_tiempo = {1: 0, 2: 0.25, 3: 0.75, 4: 1.5, 5: 2.5}
map_frecuencia = {1: 0, 2: 0.2, 3: 0.65, 4: 1.25}

# Listas de características según tipo de actividad
# Variables relacionadas con actividades digitales
tiempo_features_digital = ["S20A", "S20G", "S20H", "S20I", "S20J"]
frecuencia_features_digital = ["S21A", "S21C", "S22F"]

# Variables relacionadas con actividades culturales
tiempo_features_cultural = ["S20B", "S20C", "S20D", "S20E", "S20F"]
frecuencia_features_cultural = ["S22A", "S22B", "S22C", "S22D", "S22E"]

# Conversión de variables digitales
# Mapear las categorías numéricas a horas estimadas para cada variable de tiempo y frecuencia digital
for feature in tiempo_features_digital:
    egd10[feature + "_horas_digital"] = egd10[feature].map(map_tiempo)

for feature in frecuencia_features_digital:
    egd10[feature + "_horas_digital"] = egd10[feature].map(map_frecuencia)

# Sumar todas las horas digitales en una nueva columna unificada
# Combina tiempo y frecuencia para calcular un total de horas invertidas en actividades digitales
egd10["Total_Horas_Digital"] = egd10[
    [
        col + "_horas_digital"
        for col in digital_features
    ]
].sum(axis=1)

# Conversión de variables culturales
# Mapear las categorías numéricas a horas estimadas para cada variable de tiempo y frecuencia cultural
for feature in tiempo_features_cultural:
    egd10[feature + "_horas_cultural"] = egd10[feature].map(map_tiempo)

for feature in frecuencia_features_cultural:
    egd10[feature + "_horas_cultural"] = egd10[feature].map(map_frecuencia)

# Sumar todas las horas culturales en una nueva columna unificada
# Combina tiempo y frecuencia para calcular un total de horas invertidas en actividades culturales
egd10["Total_Horas_Cultural"] = egd10[
    [col + "_horas_cultural" for col in cultural_features]
].sum(axis=1)

Podemos observar como quedan nuestras nuevas variables unificadas:

In [ ]:
# Imprime el total de horas invertidas en actividades culturales y digitales
print(egd10[["Total_Horas_Digital"]])
print(egd10[["Total_Horas_Cultural"]])

## Características de la tarjeta de datos


El nuevo dataset generado, `tarjeta_H4`, representa una versión simplificada y procesada de los datos originales, obtenida a través de la transformación y selección de las características más relevantes para la hipotésis que nos atañe en este caso.
Las transformaciones empleada en las características de importancia seleccionadas ha sido principalmente la reducción de la dimensionalidad mediante la suma, y la conversión de tipos de datos por medio de mapeo.

In [ ]:
# Filtrar las columnas deseadas
egd10_h4 = egd10[["Total_Horas_Cultural", "Total_Horas_Digital", "Nota_Media"]]

# Guardar el nuevo DataFrame en un archivo CSV
egd10_h4.to_csv("data/tarjeta_H4.csv", index=False, float_format="%.2f")

Las variables finales del dataset son:

- `Total_Horas_Cultural`: total de horas diarias que el estudiante dedica a ocio del tipo *actividades culturales y formativas*
- `Total_Horas_Digital`: total de horas diarias que el estudiante dedica a ocio del tipo *actividades de ocio digital*
- `Nota_Media`: nota media del estudiante

Sin valores nulos ni valores por defecto.

## Líneas de Trabajo


Para validar la hipótesis de que el ocio de los estudiantes influye en su rendimiento académico, se podría realizar un análisis de correlación entre las variables que reflejan el tipo de ocio frecuentado y su tiempo de dedicación y sus calificaciones. Se podrán técnicas de clustering para identificar agrupaciones y patrones entre los datos.

Este análisis permitirá explorar la posible conexión entre ambas variables, cuantificando tanto su relevancia estadística como la intensidad de su impacto.